In [16]:
from os import getenv
from borsdata_sdk import BorsdataAPI
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.dpi'] = 200


api_key = '14e98818e51e42928578dd7e421f0e1d'
bors_api = BorsdataAPI(api_key)

In [12]:
'''report data = {'year': 2018, 'period': 5, 'revenues': 27772.0, 'gross_Income': 7049.0, 
'operating_Income': 1956.0, 'profit_Before_Tax': 1829.0, 'profit_To_Equity_Holders': 1322.0, 
'earnings_Per_Share': 5.2103, 'number_Of_Shares': 253.728, 'dividend': 1.85, 'intangible_Assets': 1999.0,
'tangible_Assets': 5468.0, 'financial_Assets': 91.0, 'non_Current_Assets': 7558.0, 'cash_And_Equivalents': 737.0, 
'current_Assets': 10477.0, 'total_Assets': 18035.0, 'total_Equity': 8996.0, 'non_Current_Liabilities': 3681.0, 
'current_Liabilities': 5358.0, 'total_Liabilities_And_Equity': 18035.0, 'net_Debt': 2667.0, 
'cash_Flow_From_Operating_Activities': 1090.0, 'cash_Flow_From_Investing_Activities': -723.0, 
'cash_Flow_From_Financing_Activities': -123.0, 'cash_Flow_For_The_Year': 244.0, 'free_Cash_Flow': 367.0, 
'stock_Price_Average': 134.95, 'stock_Price_High': 156.9, 'stock_Price_Low': 113.0, 
'report_Start_Date': '2018-01-01T00:00:00', 'report_End_Date': '2018-12-31T00:00:00', 
'broken_Fiscal_Year': False}'''

def load_report_data():
    instruments = bors_api.get_instruments()
    reports = {}
    idents = []
    tickers = []
    for instrument in instruments:
        stock = instrument.ticker
        ident = instrument.insId
        reports[ident] = bors_api.get_instrument_reports(ident, 'year')
        idents.append(ident)
        tickers.append(stock)
    return idents, tickers, reports

In [13]:
'''This is not needed to be updated on a regular basis, only once in a while'''
idents, tickers, reports = load_report_data()

In [22]:
print(reports[2][0].dividend)

1.85


In [53]:
#columns = ['id', 'ticker', 'close', 'dividend', 'earnings', 'growth_rev_5y', 'growth_ear_5y', 'growth_div_5y', 'equity_ration']
analysis_data = pd.DataFrame()


analysis_data['id'] = idents
analysis_data['ticker'] = tickers
analysis_data['close'] = None
analysis_data['dividend'] = None
analysis_data['earnings'] = None
analysis_data['growth_rev_5y'] = None
analysis_data['growth_ear_5y'] = None
analysis_data['growth_div_5y'] = None
analysis_data['equity_ratio'] = None
analysis_data['p/e'] = None
analysis_data['profit_return'] = None
analysis_data['direct_return'] = None

for i, row in analysis_data.iterrows():
    row['close'] = prices[i].c
    report = reports[int(row['id'])]
    
    if int(row['id']) in reports.keys() and len(reports[int(row['id'])])>0:
        row['dividend'] = report[0].dividend
        row['earnings'] = report[0].earnings_Per_Share
        if len(report) > 4 and float(report[4].revenues)>0 and (float(report[0].revenues)-float(report[4].revenues))>0:
            row['growth_rev_5y'] = np.power(((float(report[0].revenues)-float(report[4].revenues))/float(report[4].revenues)),0.2)
        if len(report) > 4 and float(report[4].earnings_Per_Share)>0 and (float(report[0].earnings_Per_Share)-float(report[4].earnings_Per_Share))>0:
            row['growth_ear_5y'] = np.power(((float(report[0].earnings_Per_Share)-float(report[4].earnings_Per_Share))/float(report[4].earnings_Per_Share)),0.2)
        if len(report) > 4 and float(report[4].dividend)>0 and (float(report[0].dividend)-float(report[4].dividend))>0:
            row['growth_div_5y'] = np.power(((float(report[0].dividend)-float(report[4].dividend))/float(report[4].dividend)),0.2)
        if float(report[0].total_Assets)>0:
            row['equity_ratio']  = float(report[0].total_Equity)/float(report[0].total_Assets)
    analysis_data.iloc[i] = row


   id  ticker   close dividend earnings growth_rev_5y growth_ear_5y  \
0   2     AAK  191.05     1.85   5.2103      0.884159      0.864828   
1   3     ABB   171.9    6.744   8.5921          None          None   
2   6    ACTI   41.45        0  -0.2683      0.986161          None   
3   7  ANOD B   12.05      2.5   4.8195      0.965813      0.840284   
4   8  ADDT B    3.03        5   9.8438      0.869727      0.921544   
5  10    ALFA   147.5        5  10.7734      0.692855      0.882064   
6  12    ATRE     245     1.25   0.0323          None          None   
7  13    ANOT    2.52        0  -1.0307          None          None   
8  14   ARISE   170.4        0   0.6292      0.718115          None   
9  17  ASSA B   14.75      3.5   2.4784      0.862967          None   

  growth_div_5y equity_ratio   p/e profit_return direct_return  
0      0.915877     0.498808  None          None          None  
1      0.539171      0.32704  None          None          None  
2          None     0.2

In [56]:
'''This should be updated daily'''
prices = bors_api.get_instrument_stock_price_last()

for i, row in analysis_data.iterrows():
    row['close'] = prices[i].c
    row['p/e'] = float(prices[i].c)/float(row['earnings']) if row['earnings'] is not None and float(row['earnings'])!=0 else None
    row['direct_return'] = float(prices[i].c)/float(row['dividend']) if row['dividend'] is not None and float(row['dividend'])!=0 else None
    row['profit_return'] = 1/row['p/e'] if row['p/e'] is not None else None
    analysis_data.iloc[i] = row

In [57]:
print(analysis_data.head(10))

   id  ticker   close dividend earnings growth_rev_5y growth_ear_5y  \
0   2     AAK  191.05     1.85   5.2103      0.884159      0.864828   
1   3     ABB   171.9    6.744   8.5921          None          None   
2   6    ACTI   41.45        0  -0.2683      0.986161          None   
3   7  ANOD B   12.05      2.5   4.8195      0.965813      0.840284   
4   8  ADDT B    3.03        5   9.8438      0.869727      0.921544   
5  10    ALFA   147.5        5  10.7734      0.692855      0.882064   
6  12    ATRE     245     1.25   0.0323          None          None   
7  13    ANOT    2.52        0  -1.0307          None          None   
8  14   ARISE   170.4        0   0.6292      0.718115          None   
9  17  ASSA B   14.75      3.5   2.4784      0.862967          None   

  growth_div_5y equity_ratio       p/e profit_return direct_return  
0      0.915877     0.498808   36.6678     0.0272719        103.27  
1      0.539171      0.32704   20.0068     0.0499831       25.4893  
2          